# Problemas


En ``warehouse_allocation`` se han definido diferentes problemas relativas al problemas de slotting. Cada problema es un clase que hereda de [Problem](https://pymoo.org/interface/problem.html), que es una clase abstracta en ``pymoo`` que define la estructura para poder entregar un problema *user-defined* a un solver.

Un problema básicamente define la función objetivo, y las restricciones del problema de optimización.  Hemos considera el problema de slotting en las siguientes versiones


## Representación poblacional

En ``pymoo`` la población es representada como una matriz, donde cada fila es un individuo y cada columna es una variable. En ``warehouse_allocation`` la conveción es llamar $X_{\text{pop}}$ o $X$ a dicha matriz. En nuestro problema, cada fila de la matriz representa un slotting **en su representación vectorial**. Si $Q$ es la cantidad de SKUs a alocar y $C$ es la cantidad de clusters (pensar en pasillos) donde se van a alocar los SKUs, un slotting es un vector $x\in \mathbb{R}^{QC}$ binario. Dado un SKU $j$, los valores $x[kj]$ con $k = 1,..., C$ nos informan en que clusters está alocado si el valor es $1$. 

Los algoritmos genéticos trabajan con un tamaño poblacional fijo $N$, en ese caso se tiene que $X_{\text{pop}}\in \mathbb{M}\{0,1\}_{N, QC}$, donde $\mathbb{M}\{0,1\}_{N, QC}$ son las matrices binarias de $N$ filas y $QC$ columnas.


Dado un indivudo de la población (slotting), consideramos usualmente su **representación matricial**, que consiste en una matriz $\mathbb{M}\{0,1\}_{C, Q}$, es decir, cada fila es un cluster y cada columna un SKU




## Problemas (API)

In [1]:
import numpy as np

from warehouse_allocation.models.chung_not_constraints import (
    ChungProblem,
    ChungProblemStrictlyWeighted,
    ChungProblemWithDivisions,
    ChungProblemWithDivisionsPlusWeight
)
    
# Demanda de cada uno de los skus
D = np.array([10,10,10])

# Capacidad de los clusters
Z = np.array([2,2])

# Matriz de afinidad (En este ejemplo todos los skus aparecieron en 5 ordenes simultáneamente)
OCM = np.array([[0, 5, 5], [5,0,5], [5, 5, 0]])

# Vector de pesos, cada peso corresponde a un SKU dada la indexación
W = np.array([5,5,10])

# Matriz de tolerancia de pesos, cada fila es la tolerancia de un cluster
# la primera columna es el peso mínimo que soporta, y la segunda el máximo
WT = np.array([[2,10], [0,8]])

# Problema de Chung original bi objetivo
problem_original = ChungProblem(D = D, Z = Z, OCM = OCM)

# Problema de Chung con clusters que permiten skus de acuerdo a peso.
# Es un problema de peso estricto al incorporar los pesos como una constraint
problem_strictly_weighted = ChungProblemStrictlyWeighted(D = D, Z = Z, OCM = OCM , W = W, WT = WT)

In [2]:
# Clasificación de rotación/división de cada uno de los skus
# Hay dos skus de clasificación 0 y uno de 1
DIV_TYPES = np.array([0,1,1])

# Matriz de capacidad de los clusters considerando rotación
# Cada fila corresponde a un cluster y cada columna j es la cantidad
# de skus del tipo de división j que admite
# Notar que la cantidad de columnas debe coincidir con la cantidad
# de registros únicos de DIV_TYPES
Z_ROT = np.array([[2,1], [1,2]])

# problema de Chung con restricciones de rotación
problem_division = ChungProblemWithDivisions(D = D, Z = Z_ROT, OCM = OCM, division_types = DIV_TYPES)

# problema de Chung con restricciones de rotación + peso
problem_division_plus_weight = ChungProblemWithDivisionsPlusWeight(D = D, Z = Z_ROT, W = W, WT = WT, OCM = OCM, division_types = DIV_TYPES)
    

In [3]:
# Población con 2 individuos (slottings)
X_pop = np.array([[1,0,1,0,1,0],[1,1,1,0,0,0]])

print(f"Objetivo de afinidad: {problem_original.Q(X_pop)}", f"\nObjetivo de tráfico {problem_original.W_max_pop(X_pop)}")

Objetivo de afinidad: [ -5 -15] 
Objetivo de tráfico [20 30]


Adicionalmente cada uno define el método ``constraints``

In [4]:
problem_original.constraints(X_pop)

array([[ 0. , -1. , -0.1, -0.1, -0.1],
       [ 1. , -2. , -0.1, -0.1, -0.1]])

In [5]:
problem_strictly_weighted.constraints(X_pop)

array([[ 0. , -1. , -0.1, -0.1, -0.1, -0.1, -0.1, -0.1],
       [ 1. , -2. , -0.1, -0.1, -0.1, -0.1, -0.1, -0.1]])

In [6]:
problem_division.constraints(X_pop)

array([[-1. , -1. ,  0. , -1. , -0.1, -0.1, -0.1],
       [-1. , -1. ,  1. , -2. , -0.1, -0.1, -0.1]])

In [7]:
problem_division_plus_weight.constraints(X_pop)

array([[-1. , -1. ,  0. , -1. , -0.1, -0.1, -0.1, -0.1, -0.1, -0.1],
       [-1. , -1. ,  1. , -2. , -0.1, -0.1, -0.1, -0.1, -0.1, -0.1]])

## Validaciones sobre los parámetros

Es importante el correcto uso de los problemas definidos anteriormente, para ello es responsabilidad del usuario preocuparse por la indexación y el significado de los parámetros que ingresa como input.

Se han definido validadores básicos que tienen que ver con la dimensión de los parámetros, por ejemplo, si el vector ``Z``, tiene ``len(Z) = 2`` y el vector ``len(WT) = 3``, se arrojará error



Otro ejemplo, es la matriz de afinidad ``OCM``. Esta matriz es simétrica, y para efectos del problema de optimización solo es necesaria la parte triangular superior, los problemas internamente triangulizan la matriz

In [ ]:
wrong_problem =  ChungProblemStrictlyWeighted(
                    D = D, 
                    Z = Z, 
                    OCM = OCM , 
                    W = W, 
                    WT = np.array([[2,10], [0,8], [0,2]])
                 )

In [8]:
# Matriz triangular
problem_strictly_weighted.OCM

array([[0, 5, 5],
       [0, 0, 5],
       [0, 0, 0]])

Si ``OCM`` no es simétrica, triangular superior o inferior, se arrojará error

In [ ]:
# OCM no simetrica
wrong_problem =  ChungProblemStrictlyWeighted(
                    D = D, 
                    Z = Z, 
                    OCM = np.array([[0,10,10], [0,0,10], [10,10,0]]) , 
                    W = W, 
                    WT = np.array([[2,10], [0,8], [0,2]])
                )

Hay múltiples validadores de consistencia de los parámetros, pero la responsbilidad final es de quién usa este proyecto.